In [1]:
%reload_ext autoreload
%autoreload 2
%cd ..

/Users/anton.javelosa/Desktop/Personal/Personal Projects/taxi_demand_predictor


/Users/anton.javelosa/Desktop/Personal/Personal Projects/taxi_demand_predictor/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import src.config as config

In [13]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/760982
Connected. Call `.close()` to terminate connection gracefully.


In [14]:
# create feature view if it doesn't exist yet
# This feature view only uses the features from the feature group so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print("Feature view already exists. Skip creation")

# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already exists. Skip creation


In [15]:
ts_data, __ = feature_view.training_data(
    description='Time series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (46.94s) 


In [16]:
ts_data.drop(columns=['pickup_ts'], inplace=True)
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data.head(3)

,pickup_hour,rides,pickup_location_id
113717,2022-01-01 00:00:00+00:00,0,1
1925053,2022-01-01 01:00:00+00:00,0,1
1042360,2022-01-01 02:00:00+00:00,0,1


In [17]:
print(ts_data.pickup_hour.max())
print(ts_data.pickup_hour.min())

2024-05-31 22:00:00+00:00
2022-01-01 00:00:00+00:00


In [18]:
# ignore deprecation warnings for now

import warnings
warnings.filterwarnings("ignore")

In [29]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # 1 month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 265/265 [00:29<00:00,  8.91it/s]


features_and_target.shape=(217469, 675)


In [30]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*2), utc=True)

print(f'{cutoff_date=}')

features_and_target['pickup_hour'] = pd.to_datetime(features_and_target['pickup_hour'], utc=True)

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2024-04-06 00:00:00+0000', tz='UTC')
X_train.shape=(210074, 675)
y_train.shape=(210074,)
X_test.shape=(7395, 675)
y_test.shape=(7395,)


In [31]:
import numpy as np
import optuna
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyperparameters, it trains the model and computes
    an average validation error based on a TimeSeriesSplit
    """
    # choose hyperparameters
    hyperparams = {
        'metric': 'mae',
        'verbose': -1,
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 3, 100)
    }

    tss = TimeSeriesSplit(n_splits=4)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data into train and validation sets
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index, :]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]

        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)

        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)

    # Return the mean of the validation errors
    return np.array(scores).mean()

In [32]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

[I 2024-06-01 08:57:05,026] A new study created in memory with name: no-name-f84eaad9-6295-4c82-9b73-ec719eed6114
[I 2024-06-01 08:57:25,606] Trial 0 finished with value: 3.0637990791388954 and parameters: {'num_leaves': 45, 'feature_fraction': 0.8620471043877254, 'bagging_fraction': 0.9579330366759802, 'min_child_samples': 61}. Best is trial 0 with value: 3.0637990791388954.
[I 2024-06-01 08:57:59,316] Trial 1 finished with value: 3.0373733100849414 and parameters: {'num_leaves': 119, 'feature_fraction': 0.8390014317512555, 'bagging_fraction': 0.22151986231068418, 'min_child_samples': 92}. Best is trial 1 with value: 3.0373733100849414.
[I 2024-06-01 08:58:45,811] Trial 2 finished with value: 2.98581518384926 and parameters: {'num_leaves': 189, 'feature_fraction': 0.7394280017162123, 'bagging_fraction': 0.9416956392100126, 'min_child_samples': 11}. Best is trial 2 with value: 2.98581518384926.
[I 2024-06-01 08:59:00,872] Trial 3 finished with value: 3.0996242115209167 and parameters: 

In [33]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 189, 'feature_fraction': 0.7394280017162123, 'bagging_fraction': 0.9416956392100126, 'min_child_samples': 11}


In [34]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.7394280017162123, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7394280017162123
[LightGBM] [Warning] bagging_fraction is set=0.9416956392100126, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9416956392100126
[LightGBM] [Warning] feature_fraction is set=0.7394280017162123, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7394280017162123
[LightGBM] [Warning] bagging_fraction is set=0.9416956392100126, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9416956392100126
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.279903 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 172156
[LightGBM] [Info] Number of data points in the train set: 210074, number of used features: 677
[LightGBM] [Info] Start training from sco

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x325959940>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.9416956392100126,
                               feature_fraction=0.7394280017162123,
                               min_child_samples=11, num_leaves=189))])

In [35]:
predictions = pipeline.predict(X_test)

[LightGBM] [Warning] feature_fraction is set=0.7394280017162123, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7394280017162123
[LightGBM] [Warning] bagging_fraction is set=0.9416956392100126, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9416956392100126


In [36]:
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=3.0956


In [38]:
import joblib
from src.paths import MODELS_DIR
joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['/Users/anton.javelosa/Desktop/Personal/Personal Projects/taxi_demand_predictor/models/model.pkl']

In [39]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [48]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="ny_taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

import os

model.save(os.path.join(MODELS_DIR, 'model.pkl'))

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

OSError: Could not find path /Users/anton.javelosa/Desktop/Personal/Personal Projects/taxi_demand_predictor/models/model.pkl/static in the local filesystem or in Hopsworks File System